# Queueing theory - M/M/1 example

### 1. Imports

In [1]:
import openqtsim

### 2. Prepare inputs

In [2]:
# Basic input
t_scale = 1 # 1: hours, 60: minutes, 3600: seconds, etc
Lambda = 8
Mu = 9

In [3]:
# Create Arrival and Service processes and specify the number of servers
A = openqtsim.ArrivalProcess("M", arr_rate=Lambda, t_scale=t_scale)
S = openqtsim.ServiceProcess("M", srv_rate=Mu, t_scale=t_scale)
c = 1

In [4]:
# Create the queue object and use it to create a simulating object
q = openqtsim.Queue(A, S, c)
sim = openqtsim.Simulation(q, t_scale=t_scale, seed=1)

### 3. Run the simulation

In [5]:
# set stop limits
tol = 0.0001
max_arr = 50000

In [6]:
%%time
print('Start simulation of "{}" queue'.format(q.kendall_notation))
print('')

# run simulation - simulation stops either when max arrivals is reached or the tolerance limits are achieved
sim.run(max_arr=max_arr, IAT_tol=tol, ST_tol=tol) 

Start simulation of "M/M/1/inf/inf/FIFO" queue

Nr of customers: 10777
Accuracy IAT: 9.519399568647113e-05
Tolerance: 0.0001
Accuracy ST: 9.643113860524632e-05
Tolerance: 0.0001

Wall time: 15.2 s


### 4. Analyse and visualise results

#### Get statistics

In [7]:
print('--- input statistics --------')
print("aver_IAT_in_secs: {:.4f} [hour]".format(1/Lambda))
print("aver_ST_in_secs: {:.4f} [hour]".format(1/Mu))
print('')

print('--- simulation statistics ---')
sim.get_stats()

--- input statistics --------
aver_IAT_in_secs: 0.1250 [hour]
aver_ST_in_secs: 0.1111 [hour]

--- simulation statistics ---
Average IAT: 0.1249 [hours]
Average ST: 0.1112 [hours]
Rho: System utilisation: 0.8902

Total number of customers: 10777.00

W_s: Average time spent in the system: 0.85 [hours]
W_q: Average time spent in the queue: 0.74 [hours]
Average nr arrivals: 8.00 [# per hour]
Average nr services: 8.99 [# per hour]

Total waiting time: 7957.1539 [hours]
Average waiting time of all customers: 0.74 [hours]
Average waiting time of customers that waited: 0.83 [hours]

P_0: Probability of idle server (nobody in the system): 0.1098
Probability that somebody is waiting: 0.8917
Probability that nobody is waiting: 0.1083
P_2: Probability that 1 person is waiting: 0.1051
P_3: Probability that 2 persons are waiting: 0.0886
P_4: Probability that 3 persons are waiting: 0.0766
P_10: Probability that 9 persons are waiting (10 in system): 0.0423

Average waiting time as a fraction of ST: 6.

**Analytical solution: M_M_1_inf_inf_FIFO**

    arrival rate: 8.0000 per hour
    service rate: 9.0000 per hour
    rho: 0.8889

    P_0: probability that there are 0 customers in the system = 0.1111
    P_1: probability of 1 customers in the system = 0.0988
    P_10: probability of 10 customers in the system = 0.0342

    no_q: probability that there is nobody in the queue (P_0+P_1) = 0.2099
    util_sys: system utilisation rate  (P_(n>=1)) = 0.8889
    util_ser: server utilisation rate (P_(n>=2)) = 0.7901

    L_s: the long term average number of customers present in the system = 8.0000
    L_q: the long term average number of customers present in the queue = 7.1111
    W_s: the long term average time spent in the system = 1.0000 hour (= 60.0 minutes)
    W_q: the long term average waiting time in the queue = 0.8889 hour (= 53.3 minutes)

#### Return log

In [8]:
sim.return_log()
# c = customer
# IAT = inter arrival time
# ST = service time
# AT = arrival time
# TSB = time service begins
# TSE = time service ends
# TCSS = time customer spends in the system
# TCWQ = time customer waits in the queue
# ITS = idle time of the server
# QL = queue length

,c_id,IAT,ST,AT,TSB,TSE,TCSS,TCWQ,ITS,QL
0,1,0.067451,0.000013,0.067451,0.067451,0.067463,0.000013,0.000000,0.067451,0
1,2,0.159266,0.017634,0.226717,0.226717,0.244351,0.017634,0.000000,0.159253,0
2,3,0.045002,0.022902,0.271718,0.271718,0.294620,0.022902,0.000000,0.027367,1
3,4,0.012110,0.056161,0.283828,0.294620,0.350781,0.066952,0.010791,0.000000,1
4,5,0.052997,0.060371,0.336826,0.350781,0.411152,0.074326,0.013955,0.000000,0
5,6,0.096745,0.025414,0.433571,0.433571,0.458984,0.025414,0.000000,0.022419,0
6,7,0.144485,0.003086,0.578056,0.578056,0.581141,0.003086,0.000000,0.119071,0
7,8,0.263087,0.060010,0.841143,0.841143,0.901153,0.060010,0.000000,0.260002,0
8,9,0.138760,0.016808,0.979903,0.979903,0.996711,0.016808,0.000000,0.078750,0
9,10,0.102251,0.179241,1.082154,1.082154,1.261395,0.179241,0.000000,0.085443,1


In [9]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 12))

fig.suptitle('Overview of stochastic variables')
ax1.hist(sim.log["IAT"],1000); ax1.set_title('IAT');
ax2.hist(sim.log["ST"],1000); ax2.set_title('ST');

## Sandbox

### Small test M//M/1 queue simulator

In [161]:
from scipy import stats
import numpy as np
import pandas as pd

In [162]:
def get_IAT_and_ST(lam, mu, nr_arr, seed):# generate list of inter arrival times
    
    rv_iat = stats.expon(scale = 1/lam)
    rv_st = stats.expon(scale = 1/mu)
    
    # generate list of inter arrival times
    np.random.seed(seed)
    IAT = rv_iat.rvs(nr_arr)

    # generate list of service times
    np.random.seed(seed)
    ST = rv_st.rvs(nr_arr)
    
    return IAT, ST

In [163]:
def calculate_MM1(IAT, ST):
    
    df = pd.DataFrame()
    AT = []
    TSB = []
    TSE = []
    TCSS = []
    TCWQ = []
    ITS = []
    for i in range(len(IAT)):
        if i == 0:
            AT.append(IAT[i])
        else:
            AT.append(AT[i-1]+IAT[i])
        if i == 0:
            TSB.append(AT[i])
        else:
            TSB.append(np.max([AT[i], TSE[i-1]]))
        TSE.append(TSB[i]+ST[i])   # moment service begins plus service time
        TCSS.append(TSE[i]-AT[i])  # moment of arrival until service ends
        TCWQ.append(TSB[i]-AT[i])  # moment of arrival until service begins
        if i == 0:
            ITS.append(IAT[i])  # the server will start idle until the first arrival
        else:
            ITS.append(np.max([AT[i]-TSE[i-1], 0]))  # todo: I don't think this works with multiple servers

    df["IAT"]=IAT
    df["ST"]=ST
    df["AT"]=AT
    df["TSB"]=TSB
    df["TSE"]=TSE
    df["TCSS"]=TCSS
    df["TCWQ"]=TCWQ
    df["ITS"]=ITS
    
    return df

In [197]:
def get_stats(df):
    
    value = np.mean(df["TCWQ"])/np.mean(df["ST"])
    print('Waiting time over service time: {:.2f}'.format(value))

    value = np.mean(df[df["TCWQ"]!=0])/np.mean(df["ST"])
    print('Waiting time over service time (for customers that waited): {:.2f}'.format(value))
    
    value = (df["TSE"].iloc[-1] - np.sum(df["ITS"]))/df["TSE"].iloc[-1]
    print('Rho: system utilisation: {:.4f}'.format(value))
    
    value = np.sum(df["ITS"]) / df["TSE"].iloc[-1]
    print('P_0: probability nobody in the system: {:.4f}'.format(value))


In [199]:
value = np.mean(df[df["TCWQ"]!=0])/np.mean(df["ST"])
value

IAT         0.856740
ST          0.761546
AT      45046.110727
TSB     45048.268906
TSE     45049.030453
TCSS        2.919726
TCWQ        2.158180
ITS         0.000000
dtype: float64

In [198]:
IAT, ST = get_IAT_and_ST(8, 9, 80000, 5)
df = calculate_MM1(IAT, ST)
get_stats(df)

Waiting time over service time: 1.92


TypeError: unsupported format string passed to Series.__format__